In [2]:
from pandas_datareader import data
import datetime
from datetime import timedelta
import jpholiday
import operator
import pandas as pd

#ref. : https://qiita.com/hid_tanabe/items/3c5e6e85c6c65f7b38be
def isBizDay(DATE):
    Date = datetime.date(int(DATE.year), int(DATE.month), int(DATE.day))
    if Date.weekday() >= 5 or jpholiday.is_holiday(Date):
        return 0
    else:
        return 1

def n_bizdays_ago(n): #n営業日前を出す
    today = datetime.datetime.now()
    i = n
    while True:
        if isBizDay(today - timedelta(i)): #n日前が平日になるまで遡る
            n_bizdays = today - timedelta(i)
            break
        else:
            i+=1
    return n_bizdays

def stock_pl(df_purchase): #買った日付、買った銘柄（証券コード）,買った数　の列を含むdf
    pl_sum = 0
    df_purchase_pl = df_purchase #損益用に新しいdf
    df_purchase_pl['buy'] = 0 #買った日の終値いくら　列追加
    df_purchase_pl['yesterday'] = 0 #昨日の終値いくら　列追加
    df_purchase_pl['P/L'] = 0 #昨日の終値時点での損益いくら　列追加
    #df_purchase_pl.loc['sum'] = 
    for i in range(len(df_purchase)):
        df_purchase_buy2yest = data.DataReader(df_purchase_pl['Code'][i].astype('str') + '.JP', 'stooq', datetime.datetime.strptime(df_purchase['Date'][i], "%Y-%m-%d"), n_bizdays_ago(1))
        df_purchase_pl['buy'][i] = df_purchase_buy2yest['Close'][0] * df_purchase_pl['Amount'][i]
        df_purchase_pl['yesterday'][i] = df_purchase_buy2yest.iloc[-1]['Close'] * df_purchase_pl['Amount'][i]
        df_purchase_pl['P/L'][i] = df_purchase_pl['yesterday'][i] - df_purchase_pl['buy'][i]
        pl_sum += df_purchase_pl['P/L'][i]

    print(df_purchase_pl)
    print(pl_sum)


df1 = pd.read_csv('stock_purchase_data.csv', header=0)
stock_pl(df1)


<ipython-input-2-39f57ca7ff48>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_purchase_pl['buy'][i] = df_purchase_buy2yest['Close'][0] * df_purchase_pl['Amount'][i]
<ipython-input-2-39f57ca7ff48>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_purchase_pl['yesterday'][i] = df_purchase_buy2yest.iloc[-1]['Close'] * df_purchase_pl['Amount'][i]
<ipython-input-2-39f57ca7ff48>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_p

         Date  Code  Amount      buy  yesterday    P/L
0  2023-05-15  7751     200   685400     649000 -36400
1  2023-05-15  9021     100   610100     612600   2500
2  2023-05-15  9020     100   803000     810400   7400
3  2023-05-15  7974     200  1197600    1160000 -37600
-64100


In [11]:
from pandas_datareader import data
import datetime
from datetime import timedelta
import pandas as pd

df1 = pd.read_csv('stock_purchase_data.csv', header=0)
#df1.set_index(['Date'])
print(df1)
print(df1['Date'][0])
print(df1.tail(1)['Code'])
print(df1.iloc[-1]['Code'])
print(len(df1))


         Date  Code  Amount
0  2023-05-15  7751     200
1  2023-05-15  9021     100
2  2023-05-15  9020     100
3  2023-05-15  7974     200
2023-05-15
3    7974
Name: Code, dtype: int64
7974
4
